<a href="https://colab.research.google.com/github/AliGreo/Text-Based-Projects/blob/main/finetune_gemma_2_2B_on_code_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import datasets
import torch

In [ ]:
code_dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")['train']
code_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 18612
})

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_id = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             quantization_config = bnb_config)

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer

def prompt_format(query):
    prompt = f"""<start_of_turn>user {query} <end_of_turn>
    <start_of_turn>model
    """
    return prompt

prompt = "write a python code to access open api using requests and return the output as json file"

ids = tokenizer(prompt_format(prompt), return_tensors="pt").to(model.device)


streamer = TextStreamer(tokenizer=tokenizer,
                        skip_prompt=True,
                        skip_special_tokens=True)

_ = model.generate(**ids,
                   streamer=streamer,
                   max_new_tokens=2000)

```python
    import requests
    import json

    # Replace with your actual API endpoint
    api_url = "https://api.example.com/v1/data"

    # Make the API request
    response = requests.get(api_url)

    # Check for successful response
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Write the JSON data to a file
        with open("api_response.json", "w") as outfile:
            json.dump(data, outfile, indent=4)

        print("API response saved to api_response.json")
    else:
        print(f"API request failed with status code: {response.status_code}")
```

**Explanation:**

1. **Import Libraries:**
   - `requests`: For making HTTP requests to the API.
   - `json`: For working with JSON data.

2. **API Endpoint:**
   - Replace `"https://api.example.com/v1/data"` with the actual URL of your Open API.

3. **API Request:**
   - `requests.get(api_url)` sends a GET request to the specified API endpoint.

4. **Response 

In [ ]:
# prompt ="""
# Below is an instruction that describes a task. Write a response that appropriately completes the request.
# ### Instruction: Create a function to calculate the sum of a sequence of integers.
# ### Input: [1, 2, 3, 4, 5]
# ### Output:

# """

## fine tune the gemma 2 2B model

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(r=16,
                         lora_alpha=16,
                         lora_dropout=0.01,
                         task_type='CAUSAL_LM',
                         bias="none")

lora_model = get_peft_model(model, lora_config)

lora_model.print_trainable_parameters()

trainable params: 3,194,880 || all params: 2,617,536,768 || trainable%: 0.1221


In [ ]:
code_dataset = code_dataset.rename_column("prompt", "text")

ValueError: Original column name prompt not in the dataset. Current columns in the dataset: ['instruction', 'input', 'output', 'text']

In [ ]:
code_dataset = code_dataset.train_test_split(0.2)
code_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 14889
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 3723
    })
})

In [ ]:
!pip install trl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.5 MB/s eta 0:00:00


In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

args = SFTConfig(output_dir="finetune-gemma2-2B-instruct-codeDataset",
                per_device_train_batch_size=1,
                gradient_accumulation_steps=2,
                logging_steps=300,
                max_steps= 2000,
                dataset_text_field="text",
                max_seq_length=300)

trainer = SFTTrainer(
    model=lora_model,
    train_dataset=code_dataset['train'],
    processing_class=tokenizer,
    args=args,
)

Tokenizing train dataset:   0%|          | 0/14889 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14889 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: agreu77 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss
300,0.879400
600,0.657700
900,0.617900
1200,0.605400
1500,0.611000
1800,0.591100


TrainOutput(global_step=2000, training_loss=0.6553501815795898, metrics={'train_runtime': 3722.4774, 'train_samples_per_second': 1.075, 'train_steps_per_second': 0.537, 'total_flos': 8121914272862208.0, 'train_loss': 0.6553501815795898})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1739979825.5a24db99fceb.1436.0:   0%|          | 0.00/8.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlyGreo/finetune-gemma2-2B-instruct-codeDataset/commit/fb0eef73df811af8c0c5a4144cfe61144f0294aa', commit_message='End of training', commit_description='', oid='fb0eef73df811af8c0c5a4144cfe61144f0294aa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AlyGreo/finetune-gemma2-2B-instruct-codeDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='AlyGreo/finetune-gemma2-2B-instruct-codeDataset'), pr_revision=None, pr_num=None)

## Test The model

In [ ]:
tok = AutoTokenizer.from_pretrained("AlyGreo/finetune-gemma2-2B-codeDataset")
finetuned_model = AutoModelForCausalLM.from_pretrained("AlyGreo/finetune-gemma2-2B-codeDataset",
                                                        device_map="auto",
                                                        torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/41.6M [00:00<?, ?B/s]

In [ ]:
def prompt_formating(dataset):
    dataset['test_prompt'] = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\n###instruction : {dataset['instruction']}\n###input: {dataset['input']}\n\n output: """
    return dataset
test_dataset = code_dataset['test'].map(prompt_formating)

Map:   0%|          | 0/3723 [00:00<?, ? examples/s]

In [ ]:
print(test_dataset[5]['test_prompt'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.
###instruction : Create a basic program in Python that takes in two numbers and calculates the product.
###input: int a = 5;
int b = 10;

 output: 


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation",
                     model=finetuned_model,
                     tokenizer=tok,
                     max_new_tokens=4900,
                     truncation=True)

Device set to use cuda:0


In [ ]:
def prompt(prompt):
    output = generator(prompt)
    return output[0]['generated_text']

print(prompt("create a simple function that sum a list of elements"))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


create a simple function that sum a list of elements

#include <stdio.h>
#include <stdlib.h>

int sum(int *arr, int n)
{
    int sum = 0;
    for (int i = 0; i < n; i++)
    {
        sum += arr[i];
    }
    return sum;
}

int main()
{
    int arr[] = {1, 2, 3, 4, 5};
    int n = sizeof(arr) / sizeof(arr[0]);
    int sum = sum(arr, n);
    printf("Sum of the elements is %d", sum);
    return 0;
}



In [ ]:
print(prompt("using requests access the api of spotify"))

using requests access the api of spotify.

The API is a REST API, so it is possible to use any language to make requests.

The API is available at https://api.spotify.com/

The API is free to use, but you need to register to get an access token.

The API is available in English.

The API is available in the following languages:

* English
* French
* German
* Italian
* Spanish

The API is available in the following countries:

* United States
* United Kingdom
* Canada
* Australia
* New Zealand
* Japan
* South Korea
* Singapore
* Taiwan
* Hong Kong
* Malaysia
* Thailand
* Indonesia
* Philippines
* Vietnam
* India
* Brazil
* Mexico
* Argentina
* Chile
* Colombia
* Peru
* Ecuador
* Bolivia
* Paraguay
* Uruguay
* Venezuela
* Dominican Republic
* Puerto Rico
* Jamaica
* Trinidad and Tobago
* Barbados
* Guyana
* Suriname
* St. Kitts and Nevis
* St. Lucia
* St. Vincent and the Grenadines
* Antigua and Barbuda
* Grenada
* Saint Kitts and Nevis
* Saint Lucia
* Saint Vincent and the Grenadines
* 